# Kompleksowa Analiza Przestrzenna Rynku Nieruchomości w Warszawie

## 1. Cel Projektu
Celem jest wielowymiarowa analiza rynku mieszkaniowego, wykraczająca poza proste statystyki. Projekt odpowiada na pytania:
* **Gdzie powstają "wyspy bogactwa"?** (Analiza cenowa).
* **Jak miasto "oddycha" inwestycyjnie?** (Analiza czasowa).
* **Czy opłaca się uciekać na peryferia?** (Korelacja ceny z odległością od centrum).

## 2. Użyte Technologie (Stack)
W projekcie zastosowano zaawansowane metody wizualizacji przestrzennej:
* **Folium & Plugins:** Interaktywne mapy z klastrowaniem (`MarkerCluster`), mapy ciepła (`HeatMap`) oraz animacje czasowe (`HeatMapWithTime`).
* **GeoPandas & Shapely:** Przetwarzanie danych wektorowych i operacje geometryczne.
* **Matplotlib & Seaborn:** Statystyczna analiza rozkładów (Hexbins, Regresja).

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as ctx
import numpy as np
import folium
from folium.plugins import HeatMap, HeatMapWithTime, MarkerCluster, MiniMap, Fullscreen
from shapely.geometry import Point

sns.set(style="whitegrid", palette="muted")
plt.rcParams['figure.figsize'] = (14, 9)
pd.set_option('display.max_columns', None)

print("Biblioteki załadowane. Środowisko gotowe.")

In [ ]:
FILE_NAME = 'data/apartments_pl_2023_08.csv'

try:
    print(f"Próba wczytania {FILE_NAME}...")
    df = pd.read_csv(FILE_NAME)
    df = df[df['city'].str.contains('Warszawa', case=False, na=False)].copy()
    if 'buildYear' not in df.columns and 'year' in df.columns:
        df['buildYear'] = df['year']
    print(f"Wczytano {len(df)} rekordów.")
except FileNotFoundError:
    print("Brak pliku z danymi. Sprawdź ścieżkę FILE_NAME.")
    raise

if 'price_per_sqm' not in df.columns:
    df['price_per_sqm'] = df['price'] / df['squareMeters']

pkin_coords = (52.2319, 21.0067)
df['dist_pkin_km'] = np.sqrt(
    (df['latitude'] - pkin_coords[0])**2 + (df['longitude'] - pkin_coords[1])**2
) * 111.32

df = df[(df['price_per_sqm'] > 4000) & (df['price_per_sqm'] < 70000)]
gdf = gpd.GeoDataFrame(df, geometry=[Point(xy) for xy in zip(df.longitude, df.latitude)], crs="EPSG:4326")

print(f"Dane gotowe: {len(gdf)} ofert. Średnia cena: {gdf['price_per_sqm'].mean():.0f} PLN/m²")
gdf.head(3)

<h4> Analiza Miast z najwyższą średnią wartością zainteresowania: </h4>

In [ ]:
import glob

all_files = glob.glob('data/apartments_pl_*.csv')
df_all = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)

print(f"Łącznie wczytano {len(df_all):,} rekordów z {len(all_files)} plików\n")

city_stats = df_all.groupby('city').agg(
    mean_poi=('poiCount', 'mean'),
    count=('poiCount', 'count')
).sort_values('mean_poi', ascending=False)

fig, ax = plt.subplots(figsize=(16, 8))

colors = ['#1f77b4' if count >= 500 else '#7fb3d5' if count >= 100 else '#c9dce8' 
          for count in city_stats['count']]

bars = ax.bar(range(len(city_stats)), city_stats['mean_poi'], 
              color=colors, edgecolor='black', linewidth=1.2, alpha=0.9)

ax.set_title('Średnia liczba punktów zainteresowania (POI) w miastach Polski', 
             fontsize=18, fontweight='bold', pad=20)
ax.set_xlabel('Miasto', fontsize=14, fontweight='bold')
ax.set_ylabel('Średnia wartość poiCount', fontsize=14, fontweight='bold')
ax.set_xticks(range(len(city_stats)))
ax.set_xticklabels(city_stats.index, rotation=45, ha='right', fontsize=11)
ax.grid(axis='y', alpha=0.3, linestyle='--')

for i, (city, row) in enumerate(city_stats.iterrows()):
    height = row['mean_poi']
    count = int(row['count'])
    ax.text(i, height + 2, f"n={count:,}", 
            ha='center', va='bottom', fontsize=9, fontweight='bold',
            color='darkgreen' if count >= 500 else 'orange' if count >= 100 else 'red')

from matplotlib.patches import Patch
from matplotlib.lines import Line2D
legend_elements = [
    Patch(facecolor='#1f77b4', edgecolor='black', label='≥ 500 próbek (wysoka wiarygodność)'),
    Patch(facecolor='#7fb3d5', edgecolor='black', label='100-499 próbek (umiarkowana)'),
    Patch(facecolor='#c9dce8', edgecolor='black', label='< 100 próbek (niska wiarygodność)'),
    Line2D([0], [0], marker='$n$', color='w', markerfacecolor='black', 
           markersize=12, label='n = liczba próbek w zbiorze')
]
ax.legend(handles=legend_elements, loc='upper right', fontsize=11, framealpha=0.95)

plt.tight_layout()
plt.show()

## 4. Interaktywna Eksploracja: Mapa Klastrów (Marker Cluster)

Zamiast zalewać mapę tysiącami punktów, używamy **MarkerCluster**. Punkty grupują się automatycznie przy oddalaniu, a przy przybliżaniu rozdzielają się na pojedyncze oferty.
* **Kliknij w klaster**, aby przybliżyć.
* **Najedź na pinezkę**, aby zobaczyć cenę i metraż (Tooltip).

In [ ]:
m_cluster = folium.Map(location=[52.2297, 21.0122], zoom_start=10, tiles='cartodbpositron')

Fullscreen(position='topright').add_to(m_cluster)

try:
    dzielnice = gpd.read_file('data/warszawa/dzielnice_Warszawy.shp')
    dzielnice = dzielnice.to_crs(epsg=4326)
    dzielnice_layer = folium.FeatureGroup(name='Granice dzielnic', show=True)
    for idx, row in dzielnice.iterrows():
        nazwa = row.get('nazwa', row.get('NAZWA', row.get('name', row.get('NAME', 
                row.get('nazwa_dzie', row.get('NAZWA_DZIE', f'Dzielnica {idx+1}'))))))
        folium.GeoJson(
            row.geometry,
            style_function=lambda x: {
                'fillColor': 'transparent',
                'color': "#000000",
                'weight': 2,
                'fillOpacity': 0.1
            },
            highlight_function=lambda x: {
                'fillColor': "#444444",
                'fillOpacity': 0.3
            },
            popup=folium.Popup(f"<div style='font-family: Arial; padding: 5px;'><h4 style='margin:0; font-weight:bold;'>Dzielnica: {nazwa}</h4></div>", max_width=250)
        ).add_to(dzielnice_layer)
    dzielnice_layer.add_to(m_cluster)
    print(f"Wczytano {len(dzielnice)} dzielnic Warszawy")
except Exception as e:
    print(f"Nie można wczytać dzielnic: {e}")

if 'poiCount' in gdf.columns:
    heat_poi = gdf[['latitude', 'longitude', 'poiCount']].values.tolist()
    poi_layer = folium.FeatureGroup(name='Mapa ciepła punktów zainteresowania', show=False)
    HeatMap(heat_poi, 
            radius=18, 
            blur=25, 
            max_zoom=1,
            gradient={0.2: 'blue', 0.4: 'cyan', 0.6: 'lime', 0.8: 'yellow', 1: 'red'}
    ).add_to(poi_layer)
    poi_layer.add_to(m_cluster)

marker_cluster = MarkerCluster(name="Oferty Sprzedaży (próbka)")

sample_size = min(2000, len(gdf))
gdf_sampled = gdf.sample(sample_size, random_state=42)

for idx, row in gdf_sampled.iterrows():
    color = 'green' if row['price_per_sqm'] < 12000 else 'orange' if row['price_per_sqm'] < 18000 else 'red'
    rooms = row.get('rooms', 'Brak') if pd.notna(row.get('rooms')) else 'Brak'
    build_year = int(row.get('buildYear', 0)) if pd.notna(row.get('buildYear')) else 'Brak'
    poi = int(row.get('poiCount', 0)) if pd.notna(row.get('poiCount')) else 'Brak'
    popup_html = f"""
    <div style='font-family: Arial; width: 260px;'>
        <table style='width:100%; font-size: 12px;'>
            <tr><td style='font-weight:bold;'>Cena:</td><td>{row['price']:,.0f} PLN</td></tr>
            <tr><td style='font-weight:bold;'>Metraż:</td><td>{row['squareMeters']:.1f} m²</td></tr>
            <tr><td style='font-weight:bold;'>Cena/m²:</td><td>{row['price_per_sqm']:,.0f} PLN</td></tr>
            <tr><td style='font-weight:bold;'>Pokoje:</td><td>{rooms}</td></tr>
            <tr><td style='font-weight:bold;'>Rok budowy:</td><td>{build_year}</td></tr>
            <tr><td style='font-weight:bold;'>POI:</td><td>{poi}</td></tr>
        </table>
    </div>
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=f"<b>{row['price_per_sqm']:.0f} zł/m²</b>",
        icon=folium.Icon(color=color, icon='home', prefix='fa')
    ).add_to(marker_cluster)

marker_cluster.add_to(m_cluster)

folium.LayerControl(collapsed=True).add_to(m_cluster)

legend_html = '''
<style>
.legend-container {
    position: fixed;
    bottom: 20px;
    right: 10px;
    background-color: white;
    border: 2px solid grey;
    border-radius: 5px;
    z-index: 9999;
    font-size: 12px;
    padding: 5px;
    text-align: center;
}
.legend-header {
    cursor: pointer;
    padding: 5px;
    font-weight: bold;
    background-color: #f0f0f0;
    border-radius: 3px;
    user-select: none;
    text-align: center;
}
.legend-content {
    display: none;
    text-align: center;
}
.legend-content.active {
    display: block;
}
</style>
<div class="legend-container">
    <div class="legend-header" onclick="this.nextElementSibling.classList.toggle('active')">
        Legenda
    </div>
    <div class="legend-content">
        <p style="margin: 5px 0;"><i class="fa fa-map-marker fa-2x" style="color:green"></i> < 12 000 PLN/m²</p>
        <p style="margin: 5px 0;"><i class="fa fa-map-marker fa-2x" style="color:orange"></i> 12 000 - 18 000 PLN/m²</p>
        <p style="margin: 5px 0;"><i class="fa fa-map-marker fa-2x" style="color:red"></i> > 18 000 PLN/m²</p>
        <hr style='margin: 10px 0;'>
        <p style="margin: 5px 0; font-size:11px;"><b>Uwaga:</b> Heatmapa wyswietla wszystkie dane</p>
        <p style="margin: 5px 0; font-size:11px;">Natomiast próbki tylko 2000?</p>
    </div>
</div>
'''
m_cluster.get_root().html.add_child(folium.Element(legend_html))

m_cluster

## Jaka dzielnica dla kogo?

Interaktywna analiza pokazująca, które dzielnice są najbardziej przystępne pod kątem różnych kategorii (szkoły, kliniki, transport publiczny, restauracje itp.).

**Jak korzystać:**
* Wybierz kategorię z menu warstw w prawym górnym rogu
* **Ciemniejszy kolor = mniejsza wartość = lepsza dostępność** (bliżej do obiektu)
* **Jaśniejszy kolor = większa wartość = gorsza dostępność** (dalej od obiektu)
* Kliknij w dzielnicę, aby zobaczyć szczegółowe statystyki

In [ ]:
distance_columns = ['centreDistance', 'poiCount', 'schoolDistance', 'clinicDistance', 
                   'postOfficeDistance', 'kindergartenDistance', 'restaurantDistance', 
                   'collegeDistance', 'pharmacyDistance']

available_columns = [col for col in distance_columns if col in gdf.columns]
print(f"Dostępne kategorie do analizy: {', '.join(available_columns)}")

if len(available_columns) == 0:
    print("Brak kolumn z dystansami w danych. Generuję dane syntetyczne...")
    np.random.seed(42)
    gdf['centreDistance'] = gdf['dist_pkin_km'] * 1000
    gdf['schoolDistance'] = np.random.uniform(100, 2000, len(gdf))
    gdf['clinicDistance'] = np.random.uniform(200, 3000, len(gdf))
    gdf['postOfficeDistance'] = np.random.uniform(300, 2500, len(gdf))
    gdf['kindergartenDistance'] = np.random.uniform(150, 2000, len(gdf))
    gdf['restaurantDistance'] = np.random.uniform(50, 1500, len(gdf))
    gdf['collegeDistance'] = np.random.uniform(500, 5000, len(gdf))
    gdf['pharmacyDistance'] = np.random.uniform(100, 2000, len(gdf))
    available_columns = ['centreDistance', 'schoolDistance', 'clinicDistance', 
                        'postOfficeDistance', 'kindergartenDistance', 'restaurantDistance',
                        'collegeDistance', 'pharmacyDistance']
    print("Wygenerowano syntetyczne dane dystansów")

try:
    dzielnice = gpd.read_file('data/warszawa/dzielnice_Warszawy.shp')
    dzielnice = dzielnice.to_crs(epsg=4326)
    gdf_with_district = gpd.sjoin(gdf, dzielnice, how='left', predicate='within')
    print(f"Kolumny po spatial join: {list(gdf_with_district.columns)}")
    district_name_col = None
    possible_names = ['nazwa', 'NAZWA', 'name', 'NAME', 
                     'nazwa_right', 'NAZWA_right', 'name_right', 'NAME_right',
                     'nazwa_dzie', 'NAZWA_DZIE']
    for col in possible_names:
        if col in gdf_with_district.columns:
            district_name_col = col
            print(f"Znaleziono kolumnę z nazwami dzielnic: {district_name_col}")
            break
    if district_name_col is None:
        print("Nie znaleziono kolumny z nazwami dzielnic")
        print(f"   Dostępne kolumny z 'nazwa/name': {[c for c in gdf_with_district.columns if 'nazwa' in c.lower() or 'name' in c.lower()]}")
        raise ValueError("Brak kolumny z nazwami dzielnic")
    else:
        district_stats = {}
        for category in available_columns:
            stats = gdf_with_district.groupby(district_name_col)[category].agg(['mean', 'median', 'count']).reset_index()
            stats.columns = ['nazwa', 'mean', 'median', 'count']
            district_stats[category] = stats
        print(f"\nObliczono statystyki dla {len(available_columns)} kategorii w {len(district_stats[available_columns[0]])} dzielnicach")
        m_districts = folium.Map(location=[52.2297, 21.0122], zoom_start=11, tiles='cartodbpositron')
        Fullscreen(position='topright').add_to(m_districts)
        category_labels = {
            'centreDistance': 'Odległość od centrum',
            'poiCount': 'Liczba punktów zainteresowania (więcej = lepiej)',
            'schoolDistance': 'Odległość od szkół',
            'clinicDistance': 'Odległość od przychodni',
            'postOfficeDistance': 'Odległość od poczty',
            'kindergartenDistance': 'Odległość od przedszkoli',
            'restaurantDistance': 'Odległość od restauracji',
            'collegeDistance': 'Odległość od uczelni',
            'pharmacyDistance': 'Odległość od aptek'
        }
        for category in available_columns:
            stats = district_stats[category]
            dzielnice_with_stats = dzielnice.merge(stats, left_on=district_name_col, right_on='nazwa', how='left')
            if category == 'poiCount':
                dzielnice_with_stats['normalized'] = (dzielnice_with_stats['mean'] - dzielnice_with_stats['mean'].min()) / (dzielnice_with_stats['mean'].max() - dzielnice_with_stats['mean'].min())
                colorscale = ['#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15']
            else:
                dzielnice_with_stats['normalized'] = 1 - ((dzielnice_with_stats['mean'] - dzielnice_with_stats['mean'].min()) / (dzielnice_with_stats['mean'].max() - dzielnice_with_stats['mean'].min()))
                colorscale = ['#ffffcc', '#c7e9b4', '#7fcdbb', '#41b6c4', '#1d91c0', '#225ea8', '#0c2c84']
            layer = folium.FeatureGroup(name=category_labels.get(category, category), show=(category == available_columns[0]))
            for idx, row in dzielnice_with_stats.iterrows():
                if pd.notna(row['mean']):
                    norm_val = row['normalized']
                    color_idx = int(norm_val * (len(colorscale) - 1))
                    color = colorscale[color_idx]
                    nazwa_dzielnicy = row[district_name_col] if district_name_col in row else f"Dzielnica {idx+1}"
                    if category == 'poiCount':
                        popup_text = f"""
                        <div style='font-family: Arial; padding: 8px; min-width: 200px;'>
                            <h4 style='margin:0 0 8px 0; border-bottom: 2px solid #333;'>{nazwa_dzielnicy}</h4>
                            <table style='width:100%; font-size: 11px;'>
                                <tr><td style='font-weight:bold; padding: 2px 0;'>Kategoria:</td><td>{category_labels.get(category, category)}</td></tr>
                                <tr><td style='font-weight:bold; padding: 2px 0;'>Średnia:</td><td>{row['mean']:.0f} POI</td></tr>
                                <tr><td style='font-weight:bold; padding: 2px 0;'>Mediana:</td><td>{row['median']:.0f} POI</td></tr>
                                <tr><td style='font-weight:bold; padding: 2px 0;'>Liczba ofert:</td><td>{int(row['count'])}</td></tr>
                            </table>
                        </div>
                        """
                    else:
                        popup_text = f"""
                        <div style='font-family: Arial; padding: 8px; min-width: 200px;'>
                            <h4 style='margin:0 0 8px 0; border-bottom: 2px solid #333;'>{nazwa_dzielnicy}</h4>
                            <table style='width:100%; font-size: 11px;'>
                                <tr><td style='font-weight:bold; padding: 2px 0;'>Kategoria:</td><td>{category_labels.get(category, category)}</td></tr>
                                <tr><td style='font-weight:bold; padding: 2px 0;'>Średnia:</td><td>{row['mean']:.0f} m</td></tr>
                                <tr><td style='font-weight:bold; padding: 2px 0;'>Mediana:</td><td>{row['median']:.0f} m</td></tr>
                                <tr><td style='font-weight:bold; padding: 2px 0;'>Liczba ofert:</td><td>{int(row['count'])}</td></tr>
                            </table>
                        </div>
                        """
                    folium.GeoJson(
                        row.geometry,
                        style_function=lambda x, c=color: {
                            'fillColor': c,
                            'color': '#333333',
                            'weight': 1.5,
                            'fillOpacity': 0.6
                        },
                        highlight_function=lambda x: {
                            'fillOpacity': 0.8,
                            'weight': 3
                        },
                        popup=folium.Popup(popup_text, max_width=300)
                    ).add_to(layer)
            layer.add_to(m_districts)
        folium.LayerControl(position='topright', collapsed=False).add_to(m_districts)
        legend_html = '''
        <style>
        .district-legend {
            position: fixed;
            bottom: 20px;
            left: 10px;
            background-color: white;
            border: 2px solid #333;
            border-radius: 5px;
            z-index: 9999;
            font-size: 11px;
            padding: 10px;
            max-width: 250px;
        }
        .district-legend h4 {
            margin: 0 0 8px 0;
            font-size: 13px;
            border-bottom: 1px solid #333;
            padding-bottom: 4px;
        }
        </style>
        <div class="district-legend">
            <h4>Interpretacja kolorów</h4>
            <p style="margin: 4px 0;"><strong>Dla dystansów:</strong></p>
            <p style="margin: 2px 0; font-size: 10px;"><strong>Ciemny niebieski</strong> = Najbliżej (najlepsze)</p>
            <p style="margin: 2px 0; font-size: 10px;"><strong>Jasny żółty</strong> = Najdalej (najgorsze)</p>
            <hr style="margin: 8px 0;">
            <p style="margin: 4px 0;"><strong>Dla POI:</strong></p>
            <p style="margin: 2px 0; font-size: 10px;"><strong>Ciemny czerwony</strong> = Najwięcej (najlepsze)</p>
            <p style="margin: 2px 0; font-size: 10px;"><strong>Jasny różowy</strong> = Najmniej (najgorsze)</p>
        </div>
        '''
        m_districts.get_root().html.add_child(folium.Element(legend_html))
        print("Mapa utworzona pomyślnie!")
except Exception as e:
    print(f"Błąd podczas tworzenia mapy: {e}")
    import traceback
    traceback.print_exc()
    m_districts = None

m_districts

## 5. Gdzie jest najdrożej? (Weighted Heatmap)

To nie jest zwykła mapa gęstości (pokazująca gdzie jest dużo ogłoszeń). To mapa **ważona ceną za m²**.
* **Czerwone obszary:** Niekoniecznie dużo mieszkań, ale bardzo wysokie ceny jednostkowe.
* **Niebieskie/Zielone:** Obszary tańsze.

In [ ]:
m_heat = folium.Map(location=[52.2297, 21.0122], zoom_start=11, tiles="cartodbpositron")

heat_data = gdf[['latitude', 'longitude', 'price_per_sqm']].values.tolist()

HeatMap(heat_data, 
        radius=15, 
        blur=20, 
        max_zoom=1, 
        gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(m_heat)

m_heat

## 6. Analiza Statystyczno-Przestrzenna

### A. Siatka Heksagonalna (Hexbins)
Idealna do analizy rozkładu średnich cen w ujęciu dzielnicowym, eliminująca szum pojedynczych ogłoszeń.

### B. Czy opłaca się mieszkać dalej? (Regresja)
Wykres punktowy pokazujący zależność ceny od odległości od centrum.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

gdf_metric = gdf.to_crs(epsg=3857) 

hb = ax1.hexbin(gdf_metric.geometry.x, gdf_metric.geometry.y, 
               C=gdf_metric['price_per_sqm'], 
               gridsize=35, cmap='magma', reduce_C_function=np.mean, alpha=0.9, mincnt=1)
ax1.set_title('Średnia cena za m² (Siatka Heksagonalna)', fontsize=14)
ctx.add_basemap(ax1, source=ctx.providers.CartoDB.Positron)
ax1.axis('off')
plt.colorbar(hb, ax=ax1, label='PLN / m²')

sns.regplot(x='dist_pkin_km', y='price_per_sqm', data=gdf, ax=ax2, 
            scatter_kws={'alpha':0.3, 's':10}, line_kws={'color':'red'})
ax2.set_title('Spadek ceny wraz z odległością od Centrum', fontsize=14)
ax2.set_xlabel('Odległość od Pałacu Kultury (km)')
ax2.set_ylabel('Cena za m² (PLN)')
ax2.set_ylim(0, 50000)

plt.tight_layout()
plt.show()

## 7. Podróż w czasie: Rozwój Warszawy (Timeline) ⏳

Najbardziej zaawansowana część projektu. Animacja pokazująca rok budowy budynków dostępnych w sprzedaży.
Widzimy tu **fale urbanizacji**:
1. Odbudowa centrum i Muranowa (lata 50.).
2. Wielka Płyta na Ursynowie i Bemowie (lata 70/80.).
3. Współczesny "sprawl" na Białołękę i Wilanów (lata 2000+).

*Naciśnij "Play" w lewym dolnym rogu mapy.*

In [ ]:
years = sorted(gdf['buildYear'].unique())
years = [y for y in years if y >= 1945]

heatmap_data = []
heatmap_index = []

for year in range(1950, 2025, 2):
    subset = gdf[gdf['buildYear'] <= year]
    if len(subset) > 3000:
        subset = subset.sample(3000)
    data = subset[['latitude', 'longitude']].values.tolist()
    heatmap_data.append(data)
    heatmap_index.append(str(year))

m_time = folium.Map(
    location=[52.2297, 21.0122],
    zoom_start=11,
    tiles="CartoDB positron",
    width="100%",
    height="95%",
)

hm = HeatMapWithTime(
    heatmap_data,
    index=heatmap_index,
    auto_play=True,
    radius=4,
    min_opacity=0.5,
    max_opacity=0.9,
    use_local_extrema=False
)

hm.add_to(m_time)

title_html = '''<h3 align="center" style="font-size:16px"><b>Ekspansja Budownictwa w Warszawie (1950-2024)</b></h3>'''
m_time.get_root().html.add_child(folium.Element(title_html))

m_time

## 8. Wnioski Końcowe

Projekt pozwolił na zidentyfikowanie kluczowych trendów:
1.  **Premia za lokalizację:** Wykres regresji potwierdza gwałtowny spadek cen w promieniu 5-7 km od centrum. Powyżej tej granicy ceny się spłaszczają (tzw. "plateau" na Białołęce/Ursusie).
2.  **Strefy Prestiżu:** Heatmapa cenowa ujawniła, że poza ścisłym centrum, wysokie ceny tworzą zwarte klastry na Powiślu, Starym Mokotowie i w Wilanowie.
3.  **Dynamika Rozwoju:** Animacja czasowa pokazuje wyraźne przesunięcie ciężaru inwestycyjnego na południe (Piaseczno/Józefosław) i północ (Białołęka) w ostatnich 15 latach.